<a href="https://colab.research.google.com/github/masalazarl/MetNumUN2022II/blob/main/LAB%208/Week1LUpivotingGroup5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [14]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 4
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

4

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [15]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.   ]
 [1.    1.    0.    0.   ]
 [1.    1.455 1.    0.   ]
 [1.    1.714 1.742 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16]]


# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [22]:
a1 = a.copy()
a1[1, 1] = 4

Resulting matix still has full rank, but the naive LU routine breaks down.

In [21]:
np.linalg.matrix_rank(a1)

4

In [23]:
print(a)
print(a1)

[[3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071]
 [3.    1.364 0.882 0.652]
 [3.    1.071 0.652 0.469]]
[[3.    3.    3.    3.   ]
 [3.    4.    1.364 1.071]
 [3.    1.364 0.882 0.652]
 [3.    1.071 0.652 0.469]]


In [24]:
l, u = diy_lu(a1)

print(l, u)

[[ 1.     0.     0.     0.   ]
 [ 1.     1.     0.     0.   ]
 [ 1.    -1.636  1.     0.   ]
 [ 1.    -1.929  1.148  1.   ]] [[ 3.     3.     3.     3.   ]
 [ 0.     1.    -1.636 -1.929]
 [ 0.     0.    -4.795 -5.504]
 [ 0.     0.     0.     0.066]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [25]:
def check_leading_minors_det(a):
    
    '''
    checking that all leading minors of a matrix is non-zero
    '''
    
    is_non_zero = True
    
    for k in range(a.shape[0]):
        if np.linalg.det(a[:k+1,:k+1]) == 0:
            is_non_zero = False
            break
            
    return is_non_zero

In [26]:
print(check_leading_minors_det(a1))
print(check_leading_minors_det(a))

True
True


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [11]:
def modified_diy_lu(a):
    
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    
    for j in range(N-1):
        lam = np.eye(N)
        
        perm = np.eye(N)
        pivot_pos = np.argmax(abs(u[j:,j])) + j
        perm[j],perm[pivot_pos] = perm[pivot_pos],perm[j].copy()
        u = perm @ u
        
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
    
        
        lam[j+1:, j] = gamma
        P = P @ perm
        L = L @ perm.T @ lam
    return P , P.T@L , u

In [27]:
def matr_prod(matricies):
    N = matricies[0].shape[0]
    A = np.eye(N)
    for m in matricies:
        A = A @ m
    return A

In [28]:
print(matr_prod(modified_diy_lu(a1)) - a1)

print(matr_prod(modified_diy_lu(a)) - a)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  2.220e-16 -2.220e-16]
 [ 0.000e+00  0.000e+00  0.000e+00  3.331e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17]]
[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  3.331e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17]]
